<a href="https://colab.research.google.com/github/mgmeti/language-model-course/blob/main/load_and_Save_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Setup

### Installs and imports



In [1]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.12.0


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0


11490434/11490434 [==============================] - 0s 0us/step


### Define a model

In [4]:
def create_model():
  model = tf.keras.Sequential([
      keras.layers.Dense(512, activation='relu', input_shape=(784,)),
      keras.layers.Dropout(0.2),
      keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
  
  return model

In [5]:
model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

You can use a trained model without having to retrain it, or pick-up training where you left off in case the training process was interrupted. The `tf.keras.callbacks.ModelCheckpoint` callback allows you to continually save the model both *during* and at *the end* of training.

### Checkpoint callback usage

Create a `tf.keras.callbacks.ModelCheckpoint` callback that saves weights only during training:

In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback]
          )

Epoch 1/10
32/32 [==============================] - ETA: 0s - loss: 1.1311 - sparse_categorical_accuracy: 0.6850
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 2s 27ms/step - loss: 1.1311 - sparse_categorical_accuracy: 0.6850 - val_loss: 0.6946 - val_sparse_categorical_accuracy: 0.7920
Epoch 2/10
27/32 [========================>.....] - ETA: 0s - loss: 0.4505 - sparse_categorical_accuracy: 0.8831
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 15ms/step - loss: 0.4309 - sparse_categorical_accuracy: 0.8840 - val_loss: 0.5183 - val_sparse_categorical_accuracy: 0.8410
Epoch 3/10
27/32 [========================>.....] - ETA: 0s - loss: 0.2902 - sparse_categorical_accuracy: 0.9294
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 13ms/step - loss: 0.2804 - sparse_categorical_accuracy: 0.9330 - val_loss: 0.4820 - val_sparse_categorical_accuracy: 0.8430
Epoch 4/10
30/32 [=======

In [7]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

In [8]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 1s - loss: 2.3612 - sparse_categorical_accuracy: 0.1110 - 520ms/epoch - 16ms/step
Untrained model, accuracy: 11.10%


In [9]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4050 - sparse_categorical_accuracy: 0.8680 - 187ms/epoch - 6ms/step
Restored model, accuracy: 86.80%


### Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [11]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)   # round up the number of batches to the nearest whole integer

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*n_batches
    )
model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0
          )




Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [12]:
os.listdir(checkpoint_dir)

['cp-0000.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0050.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0015.ckpt.index']

In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [15]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accirscy: {:5.2f}%".format(100 * acc))


32/32 - 0s - loss: 0.4984 - sparse_categorical_accuracy: 0.8700 - 326ms/epoch - 10ms/step
Restored model, accirscy: 87.00%


### Manually save weights

In [19]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3440 - sparse_categorical_accuracy: 0.0860 - 205ms/epoch - 6ms/step
Restored model, accuracy:  8.60%


In [20]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a `.keras` zip archive.
model.save('my_model.keras')

Epoch 1/5
32/32 [==============================] - 1s 11ms/step - loss: 1.1768 - sparse_categorical_accuracy: 0.6700
Epoch 2/5
32/32 [==============================] - 0s 8ms/step - loss: 0.4352 - sparse_categorical_accuracy: 0.8810
Epoch 3/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2940 - sparse_categorical_accuracy: 0.9260
Epoch 4/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2079 - sparse_categorical_accuracy: 0.9580
Epoch 5/5
32/32 [==============================] - 0s 8ms/step - loss: 0.1538 - sparse_categorical_accuracy: 0.9700


In [21]:
new_model = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
new_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 512)               401920    
                                                                 
 dropout_10 (Dropout)        (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4161 - sparse_categorical_accuracy: 0.8680 - 225ms/epoch - 7ms/step
Restored model, accuracy: 86.80%


32/32 [==============================] - 1s 3ms/step
(1000, 10)


In [23]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 1s 8ms/step - loss: 1.2071 - sparse_categorical_accuracy: 0.6660
Epoch 2/5
32/32 [==============================] - 0s 8ms/step - loss: 0.4283 - sparse_categorical_accuracy: 0.8710
Epoch 3/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2935 - sparse_categorical_accuracy: 0.9180
Epoch 4/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2252 - sparse_categorical_accuracy: 0.9390
Epoch 5/5
32/32 [==============================] - 0s 8ms/step - loss: 0.1539 - sparse_categorical_accuracy: 0.9660


In [24]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets	fingerprint.pb	keras_metadata.pb  saved_model.pb  variables


In [25]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 512)               401920    
                                                                 
 dropout_11 (Dropout)        (None, 512)               0         
                                                                 
 dense_23 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4486 - sparse_categorical_accuracy: 0.8580 - 371ms/epoch - 12ms/step
Restored model, accuracy: 85.80%
32/32 [==============================] - 0s 3ms/step
(1000, 10)


In [27]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Epoch 1/5
32/32 [==============================] - 1s 8ms/step - loss: 1.2097 - sparse_categorical_accuracy: 0.6460
Epoch 2/5
32/32 [==============================] - 0s 8ms/step - loss: 0.4304 - sparse_categorical_accuracy: 0.8780
Epoch 3/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2946 - sparse_categorical_accuracy: 0.9150
Epoch 4/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2285 - sparse_categorical_accuracy: 0.9360
Epoch 5/5
32/32 [==============================] - 0s 8ms/step - loss: 0.1487 - sparse_categorical_accuracy: 0.9660


In [28]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 512)               401920    
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_25 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [29]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 1s - loss: 0.4302 - sparse_categorical_accuracy: 0.8570 - 545ms/epoch - 17ms/step
Restored model, accuracy: 85.70%
